# Attention Scoring Functions

Distance functions are  slightly more expensive to compute than inner products. As such, with the softmax operation to ensure nonegative attention weights, much of the work has gone into _attention scoring functions_ alphas that are much easier to compute. 

(__I need to review this concept having into account the previous experiment__).

In [2]:
import math
import torch 
from torch import nn
from d2l import torch as d2l

## Dot Product Attention